In [5]:
using System;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;

class DefiniteIntegral
{
    public static double Solve(double a, double b, Func<double, double> function, double step, int threadsNumber)
    {
        // Проверяем, что количество потоков не превышает количество шагов разбиения
        if (threadsNumber > (int)((b - a) / step))
            threadsNumber = (int)((b - a) / step);

        // Размер одного отрезка для каждого потока
        double threadStep = (b - a) / threadsNumber;

        // Массив потоков
        Task[] tasks = new Task[threadsNumber];

        // Разделяемая переменная для суммирования результатов
        long sharedResult = 0;

        // Барьер для синхронизации потоков
        Barrier barrier = new Barrier(threadsNumber);

        // Создаем и запускаем потоки
        for (int i = 0; i < threadsNumber; i++)
        {
            int threadIndex = i;
            tasks[i] = Task.Run(() =>
            {
                // Вычисляем значение интеграла на отрезке для текущего потока
                double localResult = 0;
                double localStep = step;
                if (threadIndex == threadsNumber - 1)
                    localStep = (b - a) - threadIndex * threadStep;
                for (double x = a + threadIndex * threadStep; x < a + (threadIndex + 1) * threadStep; x += localStep)
                    localResult += (function(x) + function(x + localStep)) * localStep / 2;

                // Прибавляем результат к разделяемой переменной
                Interlocked.Add(ref sharedResult, (long)(localResult * 1e15));

                // Сигнализируем барьеру о завершении работы потока
                barrier.SignalAndWait();
            });
        }

        // Дожидаемся завершения всех потоков
        Task.WaitAll(tasks);

        // Возвращаем результат
        return Math.Round(sharedResult / 1e15);
    }
}



In [6]:
#!csharp

#r "nuget: xunit, 2.8.1"

using Xunit;

var X = (double x) => x;
var SIN = (double x) => Math.Sin(x);

double a = DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2);
double b = DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8);
double c = DefiniteIntegral.Solve(0, 10, X, 1e-6, 8);

Installed Packages xunit, 2.8.1

In [7]:
Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2), 1e-4);

In [8]:
Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8), 1e-4);

In [9]:
Assert.Equal(50, DefiniteIntegral.Solve(0, 10, X, 1e-6, 8), 1e-5);